## Initialization

In [1]:
%reload_ext dotenv
%dotenv

In [55]:
from jinja2 import Environment, FileSystemLoader
import openai
import os
import pandas as pd
import requests
import re

# Assuming the Jinja2 template file is in the same directory as this script
file_loader = FileSystemLoader('templates')
env = Environment(loader=file_loader)

# load from .env
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def generate_gpt_output(prompt_name, **kwargs):
        template = env.get_template(prompt_name)


        prompt = template.render(**kwargs)
        print(prompt)
        # Define the variables to be used in the template
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
        ]

        # run the prompt with gpt3.5
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=kwargs['temperature'] if 'temperature' in kwargs else 0.9,
            max_tokens=kwargs['max_tokens'] if 'max_tokens' in kwargs else 500,
        )

        # Print the response
        return response['choices'][0]['message']['content']


In [30]:
def camas(query, **kwargs):
    subreddit = kwargs.get('subreddit', 'travel')
    response_size = kwargs.get('response_size', 100)
    url = f"https://api.pullpush.io/reddit/comment/search?subreddit={subreddit}&q={query}&size={response_size}"
    response = requests.get(url)
    return response.json()

## Internal Knowledge

In [4]:
out = generate_gpt_output('internal_knowledge_1.jinja2', destination='Paris', trip_length='3 days', context="my wife and young children")
print(out)

I'm going on a trip to Paris for 3 days.

Give me suggestions for things to do with my wife and young children there. 
Format the output as a JSON list as follows:
[{"name":<name of thing to do 1>, "justification":<why is thing to do 1 a good option>}, {"name":<name of thing to do 2>, "justification":<why is thing to do 2 a good option>}, ...]
[
  {
    "name": "Visit the Eiffel Tower",
    "justification": "The Eiffel Tower is an iconic landmark in Paris that offers stunning panoramic views of the city. It's a great opportunity for your children to learn about its history and enjoy the view from the top."
  },
  {
    "name": "Explore the Louvre Museum",
    "justification": "The Louvre Museum houses the famous Mona Lisa painting and many other art masterpieces. It offers special activities and workshops for children, making it an educational and enjoyable experience for the whole family."
  },
  {
    "name": "Take a boat ride on the Seine River",
    "justification": "A boat ride on

In [5]:
out = generate_gpt_output('internal_knowledge_1.jinja2', destination='Hamilton', trip_length='1 week', context="older parents")
print(out)

I'm going on a trip to Hamilton for 1 week.

Give me suggestions for things to do with older parents there. 
Format the output as a JSON list as follows:
[{"name":<name of thing to do 1>, "justification":<why is thing to do 1 a good option>}, {"name":<name of thing to do 2>, "justification":<why is thing to do 2 a good option>}, ...]
[{"name": "Visit the Royal Botanical Gardens", "justification": "The Royal Botanical Gardens in Hamilton is a beautiful place to explore and enjoy nature. It offers a variety of walking trails, serene gardens, and stunning views. It's a great option for older parents who appreciate peaceful surroundings and natural beauty."},
{"name": "Explore Dundurn Castle", "justification": "Dundurn Castle is a historic mansion with beautiful grounds located in Hamilton. It provides a glimpse into the city's past and offers guided tours to learn about its history. Older parents who enjoy history and architecture will appreciate this attraction."},
{"name": "Take a cruis

In [7]:
out = generate_gpt_output('internal_knowledge_1.jinja2', destination='Miami Beach', trip_length='a weekend', context="college friends")
print(out)

I'm going on a trip to Miami Beach for a weekend.

Give me suggestions for things to do with college friends there. 
Format the output as a JSON list as follows:
[{"name":<name of thing to do 1>, "justification":<why is thing to do 1 a good option>}, {"name":<name of thing to do 2>, "justification":<why is thing to do 2 a good option>}, ...]
[{"name": "South Beach", "justification": "South Beach is one of the most popular attractions in Miami Beach, known for its beautiful sandy beaches, vibrant nightlife, and trendy restaurants and bars."},
{"name": "Art Deco Historic District", "justification": "The Art Deco Historic District is a must-visit for architecture enthusiasts, with its colorful and iconic buildings from the 1930s and 1940s."},
{"name": "Wynwood Walls", "justification": "Wynwood Walls is an outdoor street art museum showcasing vibrant and creative murals by local and international artists, making it a great place for art lovers."},
{"name": "Boat Tour of Biscayne Bay", "jus

## Review-Augmented

In [8]:
with open('reviews/miami_beach.txt', 'r') as f:
    miami_reviews = f.read()


out = generate_gpt_output('review_augmented_1.jinja2', destination='Miami Beach', trip_length='a weekend', context="college friends", reviews=miami_reviews, max_tokens=1000)
print(out)

I'm going on a trip to Miami Beach for a weekend.

Here are some reviews about Miami Beach. Use them to help create a list of things to do there:
Amazing city with diversity and energy. Appreciation of properties exceeds most of the nation, so the housing market is expensive. Schools? City has two above average elementary schools, North Beach Elem (located actually in mid-beach) and South Pointe Elem. The high school has a Intl Baccalaureate program. South Beach is a tourist mess these days with high crime and dirty streets. Mid Beach is the sleepy suburban area. North Beach is more affordable, but residents have to use private schools or finagle a way into other public schools.
Miami Beach was a wonderful experience in my life. I had the opportunity to live in a place that felt like home, with the beach, locals and people full of life.In this location you will be able to feel the true experience of living in Miami. You have food, clothing and entertainment stores on every corner. On t

In [10]:
with open('reviews/detroit.txt', 'r') as f:
    detroit_reviews = f.read()


out = generate_gpt_output('review_augmented_1.jinja2', destination='Detroit', trip_length='a week', context="my immediate family and siblings", reviews=detroit_reviews, max_tokens=1000)
print(out)

I'm going on a trip to Detroit for a week.

Here are some reviews about Detroit. Use them to help create a list of things to do there:
Most my childhood was spent in Detroit visiting family, enjoying the many parks, museums, restaurants, festivals, and Tiger games. I saw Detroit at it's best and I saw its decline. It was heartbreaking for my parents who grew up in Detroit during its heyday. Now I see Detroit through my son's eyes as he frequently visits with friends. We spent the day exploring Midtown, Campus Martius, and Belle Isle. We wanted to show my mother Detroit's comeback. I was amazing to see people out again enjoying the many cafes and restaurants. We wandered through new boutiques and most everyone we met smiled and said "hello". My heart is always with Detroit!
There is a lot to do downtown, and the city is safe by Wayne and Detroit Mercy. Great food options and they have a lot of versatile events. They recently just had a salsa party where they teach you how to dance! What

In [11]:
with open('reviews/detroit.txt', 'r') as f:
    detroit_reviews = f.read()


out = generate_gpt_output('review_augmented_1.jinja2', destination='Detroit', trip_length='a week', context="my friends in their early 20s", reviews=detroit_reviews, max_tokens=1000)
print(out)

I'm going on a trip to Detroit for a week.

Here are some reviews about Detroit. Use them to help create a list of things to do there:
Most my childhood was spent in Detroit visiting family, enjoying the many parks, museums, restaurants, festivals, and Tiger games. I saw Detroit at it's best and I saw its decline. It was heartbreaking for my parents who grew up in Detroit during its heyday. Now I see Detroit through my son's eyes as he frequently visits with friends. We spent the day exploring Midtown, Campus Martius, and Belle Isle. We wanted to show my mother Detroit's comeback. I was amazing to see people out again enjoying the many cafes and restaurants. We wandered through new boutiques and most everyone we met smiled and said "hello". My heart is always with Detroit!
There is a lot to do downtown, and the city is safe by Wayne and Detroit Mercy. Great food options and they have a lot of versatile events. They recently just had a salsa party where they teach you how to dance! What

# Destination Comparison

## Internal Knowledge

In [12]:
out = generate_gpt_output('comparative_example_1.jinja2', destination='Berlin', trip_length='3 days', context="coworkers")
print(out)

I am interested in going on a trip to Berlin with coworkers for 3 days.

Give me a list of alternative destinations for Berlin and justifications for each.
Format the output as a JSON list as follows:
[{"name":<name of destination 1>, "justification":<why is destination 1 a good option>}, {"name":<name of destination 2>, "justification":<why is destination 2 a good option>}, ...]
[
  {
    "name": "Amsterdam",
    "justification": "Amsterdam is a great alternative destination to Berlin because it offers a vibrant and diverse cultural scene, beautiful canals, and iconic landmarks like the Anne Frank House and Van Gogh Museum."
  },
  {
    "name": "Prague",
    "justification": "Prague is another excellent alternative to Berlin as it is known for its stunning architecture, rich history, and charming old town. The city offers a unique blend of Gothic, Baroque, and Renaissance buildings."
  },
  {
    "name": "Barcelona",
    "justification": "Barcelona is a fantastic choice for a trip wi

# POIs with Aspect Generation

In [102]:
destination='Berlin'
trip_length='4 days'
context="friends"

out = generate_gpt_output('aspect_splitting_1.jinja2', destination=destination, trip_length=trip_length, context=context, min_factors=4, max_factors=6)
print(out)

I am interested in going on a trip to Berlin with friends for 4 days.

Give a list of 4-6 factors of consideration (such as cost, safety, etc.) for points of interests that I should consider and justification.
Format the output as a JSON list as follows:
[{"type":<type of POI 1>, "justification":<why is POI 1 a good option>}, {"type":<type of POI 2>, "justification":<why is POI 2 a good option>}, ...]
[{"type":"Cost", "justification":"Consider the cost of visiting different points of interest in Berlin. Opt for attractions that offer good value for money and fit within your budget."},
{"type":"Safety", "justification":"Ensure the points of interest you choose are located in safe areas of Berlin. Look for places that have a low crime rate and are well-lit and well-maintained."},
{"type":"Historical Significance", "justification":"Berlin has a rich history, so prioritize visiting attractions that offer historical significance. Explore landmarks such as the Berlin Wall, Brandenburg Gate, 

In [103]:
aspects = eval(out)

# print out as pandas table
aspects_df = pd.DataFrame(aspects)
aspects_df

,type,justification
0,Cost,Consider the cost of visiting different points...
1,Safety,Ensure the points of interest you choose are l...
2,Historical Significance,"Berlin has a rich history, so prioritize visit..."
3,Cultural Experience,Immerse yourself in Berlin's vibrant culture b...
4,Local Cuisine,Try out Berlin's local cuisine by visiting res...
5,Entertainment,Berlin offers a wide range of entertainment op...


In [104]:
results = {}
for aspect in aspects:
    # query = destination + ' ' + trip_length + ' ' + context + ' ' + aspect["type"]
    query = destination + ' ' + aspect["type"]

    reddit_results = camas(query, subreddit='travel', response_size=100)
    reviews = [r['body'] for r in reddit_results['data']]

    smaller_reviews = []
    # keep adding reviews until 2000 chars
    for r in reviews:
        if len(' '.join(smaller_reviews)) < 1000:
            smaller_reviews.append(r)
        else:
            break
    
    out = generate_gpt_output('aspect_review_augmented_poi_1.jinja2', destination=destination, trip_length=trip_length, context=context, reviews=smaller_reviews, aspect=aspect["type"], max_tokens=2000)
    print(aspect["type"], out)
    
    aspect_recs = []
    try:
        # use regex to parse out the list of recommendations
        out_strip = ''.join(out.splitlines())
        out_regex = re.search(r'\[(.*)\]', out_strip).group(1)
        out_regex = '[' + out_regex + ']'
        aspect_recs = eval(out_regex)
    except Exception as e:
        print('LLM output parse error: ', e)

    results[aspect["type"]] = aspect_recs

I'm going on a trip to Berlin for 4 days.

Here are some reviews about Berlin for the following consideration: Cost. Use them to help create a list of things to do there:
["RM120 for a return pass is a good deal because you have to pay more for 2 single trip tickets. A return trip Grab ride to the airport will cost you more than RM160 with toll charges. I don't understand why do you regret on having discounts on your ticket?\n\nI am not  KL native and haven't used any in recent times but here are my thoughts.\n\nKL does not have a rail pass, it uses a credit top up card like the Tough and Go card that can be use for parking, buses, trains and even buying stuffs in shops. What you've purchased was a return trip card which offer like 20-30% off from a normal single one way ticket. The railway also have a monthly user pass for daily commuters. \n\nThe cost of each trip is already cheaper than most country on the planet. And the facilities are way better than the one I've ridden in Madrid,

In [105]:
# reformat results so that keys are the aspects and values are strings
summaries = {}
for k, v in results.items():
    # each v is a list of dicts with name, justification, pros and cons
    # we want to make a string out of each dict
    summary_str = ''
    for poi in v:
        summary_str += poi['name'] + '\nJustification:\n' + poi['justification'] + '\nPros:\n' + poi['pros'] + '\nCons:\n' + poi['cons'] + '\n\n'
    summaries[k] = summary_str

columns = summaries.keys()
row0 = [summaries[k] for k in columns]

df = pd.DataFrame([row0], columns=columns)
print(df.to_markdown())

|    | Cost                                                                                                                                                                              | Safety                                                                                                                                                                                               | Historical Significance                                                                                                                                                                                                                              | Cultural Experience                                                                                                                                                                                                                       | Local Cuisine                                                                                                                      